<img style="float: left;padding: 1.3em" src="https://indico.in2p3.fr/event/18313/logo-786578160.png">  

#  Gravitational Wave Open Data Workshop #2


#### Tutorial 1.1: Discovering open data from GW observatories

This notebook describes how to discover what data are available from the [Gravitational-Wave Open Science Center (GWOSC)](https://www.gw-openscience.org).
    
[Click this link to view this tutorial in Google Colaboratory](https://colab.research.google.com/github/gw-odw/odw-2019/blob/master/Day_1/Tuto 1.1 Discovering Open Data.ipynb)

## Software installation  (execute only if running on a cloud platform or haven't done the installation yet!)

First, we need to install the software, which we do so using the `requirements.txt` file provided for this workshop:

In [0]:
import sys
!curl --silent --output requirements.txt https://raw.githubusercontent.com/gw-odw/odw-2019/master/requirements.txt
!{sys.executable} -m pip install -q -r requirements.txt

     |████████████████████████████████| 17.3MB 2.8MB/s 
     |████████████████████████████████| 29.9MB 1.4MB/s 
     |████████████████████████████████| 1.3MB 49.3MB/s 
     |████████████████████████████████| 3.0MB 36.4MB/s 
     |████████████████████████████████| 1.6MB 38.6MB/s 
     |████████████████████████████████| 13.5MB 38.0MB/s 
     |████████████████████████████████| 6.1MB 37.7MB/s 
     |████████████████████████████████| 24.8MB 95.9MB/s 
     |████████████████████████████████| 1.6MB 42.4MB/s 
     |████████████████████████████████| 51kB 21.9MB/s 
     |████████████████████████████████| 471kB 41.3MB/s 
     |████████████████████████████████| 798kB 56.2MB/s 
     |████████████████████████████████| 92kB 30.0MB/s 
     |████████████████████████████████| 61kB 24.7MB/s 
     |████████████████████████████████| 2.3MB 31.4MB/s 
     |████████████████████████████████| 102kB 35.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatib

**Important:** With Google Colab, you may need to restart the runtime after running the cell above.

In general, we could install just what we need for this tutorial using `pip` as follows:

In [0]:
!{sys.executable} -m pip install "gwosc"

/bin/bash: {sys.executable}: command not found


<div class="alert alert-info">Here we use <code>{sys.executable} -m pip install</code>, which is just a fancy way of saying <code>pip install ...</code></div>

## Querying for event information

The gwosc.datasets module provides tools to search for datasets, including filtering on GPS times.

For example, we can search for what event datasets are available:

In [0]:
from gwosc.datasets import find_datasets
events = find_datasets(type="event")
print(events)

['151008', '151012A', '151116', '161202', '161217', '170208', '170219', '170405', '170412', '170423', '170616', '170630', '170705', '170720', 'GW150914', 'GW151012', 'GW151226', 'GW170104', 'GW170608', 'GW170729', 'GW170809', 'GW170814', 'GW170817', 'GW170818', 'GW170823']


As of O2, there are 25 event datasets available; 11 of those are confident detections (with the `'GW'` prefix), the other are _marginal_ detections. For more details, see [GWTC-1](https://www.gw-openscience.org/GWTC-1/). 

`find_datasets` also accepts a `detector` keyword to return only those datasets that include data for that detector:

In [0]:
print(find_datasets(type="event", detector="V1"))
print(find_datasets(type="event", detector="G1"))

['GW170729', 'GW170809', 'GW170814', 'GW170817', 'GW170818']
['GW170817']


`find_datasets` also accepts a `segment` keyword to narrow results based on GPS time:

In [0]:
print(find_datasets(type="event", detector="L1", segment=(1164556817, 1187733618)))

['161202', '161217', '170208', '170219', '170405', '170412', '170423', '170616', '170630', '170705', '170720', 'GW170104', 'GW170608', 'GW170729', 'GW170809', 'GW170814', 'GW170817', 'GW170818', 'GW170823']


Using `gwosc.datasets.event_gps`, we can query for the GPS time of a specific event:

In [0]:
from gwosc.datasets import event_gps
gps = event_gps('GW170817')
print(gps)

1187008882.4


<div class="alert alert-info">All of these times are returned in the GPS time system, which counts the number of seconds that have elapsed since the start of the GPS epoch at midnight (00:00) on January 6th 1980. GWOSC provides a <a href="https://www.gw-openscience.org/gps/">GPS time converter</a> you can use to translate into datetime, or you can use <a href="https://gwpy.github.io/docs/stable/time/"><code>gwpy.time</code></a>.</div>

We can query for the GPS time interval for an observing run:

In [0]:
from gwosc.datasets import run_segment
print(run_segment('O1'))

(1126051217, 1137254417)


## Querying for data files

The `gwosc.locate` module provides a function to find the URLs of data files associated with a given dataset.

For event datasets, one can get the list of URLs using only the event name:

In [0]:
from gwosc.locate import get_event_urls
urls = get_event_urls('GW150914')
print(urls)

['https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5', 'https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/L-L1_GWOSC_4KHZ_R1-1126257415-4096.hdf5']


By default, this function returns all of the files associated with a given event, which isn't particularly helpful. However, we can can filter on any of these by using keyword arguments, for example to get the URL for the 32-second file for the LIGO-Livingston detector:

In [0]:
urls = get_event_urls('GW150914', duration=32, detector='L1')
print(urls)

['https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5']


# Exercises

Now that you've seen examples of how to query for dataset information using the `gwosc` package, please try and complete the following exercies using that interface:

- How many months did S6 last?
- How many events were detected during O1?
- What file URL contains data for V1 4096 seconds around GW170817?